### Libaries

In [2]:
import robotic as ry
import numpy as np
import time
from scipy.interpolate import Rbf
from scipy.interpolate import CubicSpline
print('version:', ry.__version__, ry.compiled())

version: 0.1.3 compile time: Dec 15 2023 11:24:49


### Initalise configuration

In [3]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.view()

0

### Define color map

In [4]:
color_map = [[1,0,.5], # pink
             [0.5,1,0], # lime green
             [.5,1,1], # light turqois
             [0.5,0,1], # violet
             [1,0,0], # red
             [0,1,0], # green
             [0,0,1], # blue
             [1,.5,0], # orange
             [1,1,.5], # lime yellow
             [1,.5,1], # light pink                            
]

### Gripper functions

In [5]:
def gripper_open(C, bot, widht = 0.75, speed = 0.2):
    bot.gripperMove(ry._left, widht, speed)

    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)

def gripper_close(C, bot, widht = 0.01, speed = 0.2):
    bot.gripperMove(ry._left, widht, speed)

    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)

### Create N number of building cuboids

In [6]:
def create_n_boxes(C, N, position='fixed'):

    for i in range(N):
        box_name = 'box{}'.format(i + 1)

        if position == 'fixed':
            position_val1 = 0.1 * (i - 1)
            C.addFrame(box_name) \
                .setPosition([position_val1, 0.35, 0.70]) \
                .setShape(ry.ST.ssBox, size=[0.05, 0.05, 0.10, 0.005]) \
                .setColor(color_map[i % len(color_map)]) \
                .setContact(True) \
                .setMass(1e-2)


        elif position == 'random':
            C.addFrame(box_name) \
                .setPosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), 0.70]) \
                .setShape(ry.ST.ssBox, size=[0.05, 0.05, 0.10, 0.005]) \
                .setColor(color_map[i % len(color_map)]) \
                .setContact(True) \
                .setMass(1e-2)
    
    return C

### Define construction site

In [7]:
def create_construction_site(C, position = 'fixed'):
    
    if position == 'fixed':
        target_location = C.addFrame('construction_site', 'table')
        target_location.setShape(ry.ST.box, size = [.275, .275, .1])
        target_location.setRelativePosition([-.5, -.225, .0])
        target_location.setColor([1., 0.75, 0.])
        return target_location
    
    elif position == 'random':
        target_location = C.addFrame('construction_site', 'table')
        target_location.setShape(ry.ST.box, size = [.3, .3, .1])
        target_location.setRelativePosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), .0])
        target_location.setColor([1, 0.75, 0])
        return target_location
        

In [32]:
def create_pillar_target_postion(C, position = 'fixed', n_pillars = 2):
    
    if position == 'fixed':
        for i in range(0, n_pillars):
            target_pillar = C.addFrame('pillar_target_pos' + str(i+1), 'table')
            target_pillar.setShape(ry.ST.box, size=[.065, .065, .105])
            target_pillar.setRelativePosition([-.45 - (i / 10) , -.225, .0])
            target_pillar.setColor([0,1,0])
    
    elif position == 'random':
        for i in range(0, n_pillars):
            C = C.addFrame('pillar_target_pos' + str(i+1), 'table')
            target_pillar.setShape(ry.ST.box, size=[.065, .065, .105])
            target_pillar.setRelativePosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), .0])
            target_pillar.setColor([0,1,0])
        return C
        

### Define Komo problem

In [9]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C = create_n_boxes(C = C, N = 3, position = 'fixed')
goal = create_construction_site(C, position = 'fixed')
pillar_goal = create_pillar_target_postion(C, position = 'fixed')

C.view()

0

In [10]:
def gripper_to_block(C, bot, block = 'box1', prints = False):
    qHome = C.getJointState()

    komo = ry.KOMO(C, 1, 10, 1, False)
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', block], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', block], ry.OT.eq, [1e1], [0])
    komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', block], ry.OT.eq, [1e1], [0])
    komo.addObjective([], ry.FS.distance, ['l_palm', block], ry.OT.ineq, [1e1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()

    for t in range(q.shape[0]):
        bot.moveTo(q[t])
        while bot.getTimeToEnd() > 0:
            bot.sync(C, .5)

In [28]:
def move_gripper_to(bot, add_frame_obj_str = 'target', carry_item = 'box1', prints=False):
    komo = ry.KOMO(C, 4, 1, 1, True)

    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e1])

    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', carry_item], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    komo.addObjective([], ry.FS.distance, ['l_palm', carry_item], ry.OT.ineq, [1e1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()
    
    bot.moveTo(q[-1])
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .1)

In [17]:
def generate_waypoint_guides(C, start='', target='', offset=0, n_waypoints=5, interpolation='linear'):
    start_frame = C.getFrame(start)
    start_pos = np.array(start_frame.getPosition())
    target_frame = C.getFrame(target)
    target_pos = np.array(target_frame.getPosition())
    target_pos[2] += offset


    if interpolation == 'linear':
        t_vals = np.linspace(0, 1, n_waypoints)
        x_spline = Rbf([0, 1], [start_pos[0], target_pos[0]], function='linear')
        y_spline = Rbf([0, 1], [start_pos[1], target_pos[1]], function='linear')
        z_spline = Rbf([0, 1], [start_pos[2], target_pos[2]], function='linear')

        positions = np.column_stack([x_spline(t_vals), y_spline(t_vals), z_spline(t_vals)])
    
    elif interpolation == 'cubic':
        t_vals = np.linspace(0, 1, n_waypoints)
        x_spline = Rbf([0, 1], [start_pos[0], target_pos[0]], function='linear')
        y_spline = Rbf([0, 1], [start_pos[1], target_pos[1]], function='linear')
        z_spline = Rbf([0, 1], [start_pos[2], target_pos[2]], function='cubic')
        
        positions = np.column_stack([x_spline(t_vals), y_spline(t_vals), z_spline(t_vals)])


    for i, pos in enumerate(positions):
        way_name = 'way' + str(i+1)
        C.addFrame(way_name).setShape(ry.ST.marker, [.1, .1, .1]).setPosition(pos).setColor([1, .5, 1])

    return C


In [13]:
def move_to_waypoint_guides(bot, carry_item = 'box1', n_waypoints = 5,  prints=False):
    
    komo = ry.KOMO(C, 1, 10, 1, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    
    for i in range(n_waypoints):
        komo.addObjective([i+1], ry.FS.positionDiff, ['l_gripper', 'way' + str(i+1)], ry.OT.eq, [1e1])

    komo.addControlObjective([], 0, 1e-1)
    komo.addControlObjective([], 1, 1e0)
    #komo.addObjective([], ry.FS.positionDiff, ['l_gripper', carry_item], ry.OT.eq, [1e1])
    #komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.distance, ['l_palm', carry_item], ry.OT.ineq, [1e-4])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()

    for t in range(q.shape[0]):
        bot.moveTo(q[t])
        while bot.getTimeToEnd() > 0:
            bot.sync(C, .1)

In [14]:
C = generate_waypoint_guides(C, start = 'box1', target = 'pillar_target_pos2', offset=0.115, n_waypoints=5, interpolation='linear')
C.view()

0

In [23]:
bot = ry.BotOp(C, useRealRobot = False)

## First block

In [30]:
gripper_open(C, bot)
gripper_to_block(C, bot, block='box1')
gripper_close(C, bot)
move_gripper_to(bot, add_frame_obj_str = 'way2')
move_gripper_to(bot, add_frame_obj_str = 'way3')
move_gripper_to(bot, add_frame_obj_str = 'way4')
move_gripper_to(bot, add_frame_obj_str = 'way5')
gripper_open(C, bot)



### Second Block

In [ ]:
C = generate_waypoint_guides(C, start = 'box2', target = 'pillar_target_pos1', offset=0.115, n_waypoints=5, interpolation='linear')
C.view()

In [31]:
gripper_to_block(C, bot, block='box2')
#gripper_close(C, bot)
#move_gripper_to(bot, add_frame_obj_str = 'way2')
#move_gripper_to(bot, add_frame_obj_str = 'way3')
#move_gripper_to(bot, add_frame_obj_str = 'way4')
#move_gripper_to(bot, add_frame_obj_str = 'way5')
#gripper_open(C, bot)



====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):40.246 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    28.5651  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    17.6449  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    8.19059  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    1.87223  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):   0.379939  ACCEPT
--newton-- it:   6  |Delta|:   0.163349  alpha:          1  evals:   7  f(y):  0.0318098  ACCEPT
--newton-- it:   7  |Delta|:  0.0218945  alpha:          1  evals:   8  f(y):  0.0275063  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0  evals:   8  A(x):  0.0275063  f:     0.0212  g:  0.0552421  h:  0.0906049  |x-x'|:   0.891502 	st

In [21]:
move_to_waypoint_guides(bot, carry_item='box1', n_waypoints = 3)

-- WARNING:switch.cpp:conv_times2steps:39(-1) beyond the time!: endTime=3 phases=1
[[ 0.15568491  0.28087283  0.26325646 -2.49356872 -0.13284906  2.70456271
  -0.38879101]
 [ 0.15556901  0.27794743  0.26266226 -2.49182003 -0.13200863  2.70020892
  -0.38935276]
 [ 0.1556241   0.27577788  0.26233345 -2.49051971 -0.13128588  2.69645191
  -0.38981981]
 [ 0.15585917  0.27435612  0.26226798 -2.4896466  -0.13067136  2.69324291
  -0.39020017]
 [ 0.15628477  0.27368034  0.26246154 -2.48918265 -0.13015556  2.69054006
  -0.39050226]
 [ 0.15691194  0.27375633  0.26290875 -2.48911269 -0.12973001  2.68830933
  -0.39073443]
 [ 0.15775173  0.27459765  0.26360421 -2.489424   -0.12938748  2.68652432
  -0.39090481]
 [ 0.15881345  0.27622558  0.26454275 -2.49010585 -0.12912206  2.68516653
  -0.39102203]
 [ 0.16010514  0.27867069  0.26572037 -2.49114763 -0.12892905  2.68422752
  -0.39109462]
 [ 0.16163249  0.28197303  0.26713404 -2.49253775 -0.12880468  2.68370861
  -0.39112923]]
(10, 7)
====nlp==== method

### Clear config

In [87]:
C.clear()